In [ ]:
import healpy
from cora.util import hputil
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import ticker
from ch_util import ephemeris as ephem, andata
from caput.time import unix_to_skyfield_time
import h5py
import time
from glob import glob

%load_ext autoreload
%autoreload 2

from continuum_beam import *

%matplotlib inline
plt.rcParams.update({'figure.figsize': (16, 12), 'font.size': 20})

# Try fitting beam using Haslam

In [ ]:
plot_path = "../plots/continuum_beam/"

## Select a quiet region in RA

In [ ]:
# RA range during nighttime in Sept/October in between bright point source
target_RA = (30., 60.)
#target_RA = (269., 271.)

In [ ]:
# Look at an acquisition I was using for ring maps
acq_path = "/mnt/gong/archive/20180921T004045Z_chimeN2_corr/"
gain_file = "/mnt/recv1/calibration/gain_updates/gain_20180921T050356.984083Z_cyga.h5"

In [ ]:
# get N2 frequencies
with h5py.File(acq_path + "00000000_0000.h5") as fh:
    freq = fh['index_map/freq']['centre']
    start_time = fh['index_map/time']['ctime'][0]
# read gains
with h5py.File(gain_file) as fh:
    gain = fh['gain'][:]
    gain_freq_ind = [ list(fh['index_map/freq']['centre']).index(f) for f in freq ]

In [ ]:
# choose a frequency
f_ind = 9
print freq[f_ind]

In [ ]:
target_time = [ ephem.transit_times(r, start_time) for r in target_RA ]

In [ ]:
datar = andata.CorrReader(sorted(glob(acq_path + "*.h5"))[:18])

In [ ]:
# select time and freq
datar.select_freq_physical((freq[f_ind],))
datar.select_time_range(*target_time)

In [ ]:
# select intracyl baselines for A
prod_sel = np.zeros(256*257/2, dtype=int)
for i in range(256):
    row_l = 256 - i
    start_i = i*256 - i*(i-1)/2
    full_start_i = i*2048 - i*(i-1)/2
    prod_sel[start_i:start_i + row_l] = full_start_i + np.arange(row_l)
datar.prod_sel = prod_sel

In [ ]:
data = datar.read()

In [ ]:
test_prods = np.zeros((256,256))
for p in data.prod:
    test_prods[p['input_a'], p['input_b']] += 1.

In [ ]:
plt.imshow(test_prods)

In [ ]:
# plot autos
p = 0
for i in range(256):
    plt.plot(np.abs(data.vis[0,p,:]))
    p += 256 - i

## Apply gain and exclude flagged inputs

In [ ]:
vis_cal = data.vis[0,:,:] * np.outer(gain[gain_freq_ind[f_ind],:256], 
                              gain[gain_freq_ind[f_ind],:256].conj())[np.triu_indices(256)][:,np.newaxis]

In [ ]:
wgt_cal = data.weight[0,:,:] / np.abs(np.outer(gain[gain_freq_ind[f_ind],:256], 
                              gain[gain_freq_ind[f_ind],:256].conj())[np.triu_indices(256)][:,np.newaxis])**2

In [ ]:
# plot autos
p = 0
autos_ind = []
for i in range(256):
    plt.plot(np.abs(vis_cal[p,:]))
    autos_ind.append(p)
    p += 256 - i

In [ ]:
ns_baselines = ((data.prod['input_a'] % 256).astype(float)
                 - data.prod['input_b'] % 256) * 0.3

In [ ]:
bad_input = np.prod(data.flags['inputs'], axis=1)
print np.sum(bad_input)
bad_input *= gain[gain_freq_ind[f_ind],:256] != 0
print np.sum(bad_input)
bad_prod = np.outer(bad_input, bad_input)[np.triu_indices(256)]
# remove autos
bad_prod[np.array(autos_ind)] = 0.
# remove short baselines
#bad_prod[np.where(np.abs(ns_baselines) < 3. * 0.3)[0]] = 0.
good_prod = np.where(bad_prod != 0.)[0]

In [ ]:
# remove excluded products
vis_cal = vis_cal[good_prod,:]
wgt_cal = wgt_cal[good_prod,:]
ns_baselines = ns_baselines[good_prod]

## Try on a single RA

In [ ]:
test_ind = vis_cal.shape[-1] / 2
#test_ind = vis_cal.shape[-1] - 1

In [ ]:
test_model = ModelVis(freq=freq[f_ind])

In [ ]:
test_model.set_baselines(ns_baselines)

In [ ]:
max_za = 80.
num_pix = int(2 * max_za / 2)  # approx 2 deg resolution for Haslam
#time_slice = slice(test_ind-5, test_ind+5)
time_slice = slice(test_ind-200, test_ind+200, 10)

In [ ]:
ct_offset = np.sum(vis_cal[:,:]*wgt_cal[:,:], axis=1) / np.sum(wgt_cal[:,:], axis=1)

In [ ]:
beam_sol = test_model.fit_beam(data.time[time_slice], vis_cal[:,time_slice],
                               wgt_cal[:,time_slice],
                               num_pix, max_za=max_za, rcond=1e-2)

In [ ]:
beam_sol_single = test_model.fit_beam(data.time[test_ind:test_ind+1], vis_cal[:,test_ind:test_ind+1],
                               np.ones_like(wgt_cal[:,test_ind:test_ind+1]),
                               num_pix, max_za=max_za, rcond=1e-9)

In [ ]:
plt.plot(np.linspace(-max_za, max_za, num_pix), beam_sol)
plt.plot(np.linspace(-max_za, max_za, num_pix), beam_sol_single)

In [ ]:
#plt.imshow(np.linalg.inv(test_model.M))
#plt.imshow(np.log10(np.abs(test_model.M)))
plt.imshow(np.log10(np.abs(np.dot(np.linalg.inv(test_model.M), test_model.M))),
          extent=(-max_za,max_za,-max_za,max_za))
plt.colorbar()

In [ ]:
for k in range(0, 256, 64):
    plt.plot(test_model.M[k,:], label="{}".format(k))
plt.legend()
plt.ylim(ymax=1e12, ymin=-1e12)

In [ ]:
plt.imshow(test_model.M - test_model.M.T)
plt.colorbar()

In [ ]:
U, S, V = np.linalg.svd(test_model.M)

In [ ]:
S.shape

In [ ]:
plt.plot(np.log10(S / S[0]))
plt.ylabel("$\log_{10}(\Sigma_i)$")
plt.xlabel("$i$")

In [ ]:
plt.plot(test_model.v)

In [ ]:
ns_bl_norm = np.zeros(data.vis.shape[1])
start_i = 0
for i in range(256):
    ns_bl_norm[start_i:start_i+256-i] = (np.arange(256-i) + 1)
    start_i += 256 - i
ns_bl_norm = ns_bl_norm[good_prod]

In [ ]:
test_norm = np.zeros((256,256))
test_norm[np.triu_indices(256)] = ns_bl_norm
plt.imshow(test_norm)

In [ ]:
test_vis = test_model.get_vis(data.time[test_ind:test_ind+1], vis_cal[:,test_ind:test_ind+1],
                               num_pix, max_za=max_za)
test_za = np.radians(np.linspace(-max_za, max_za, num_pix))
test_map = np.dot(test_vis[:,0]/ns_bl_norm, np.exp(-2j * np.pi * ns_baselines[:,np.newaxis]
                                        / test_model.wl * np.sin(test_za)[np.newaxis,:]))

In [ ]:
test_map_vis = np.dot(vis_cal[:,test_ind] / ns_bl_norm, np.exp(-2j * np.pi * ns_baselines[:,np.newaxis]
                                        / test_model.wl * np.sin(test_za)[np.newaxis,:]))

In [ ]:
plt.subplot(2,1,1)
plt.plot(test_za/np.pi, test_map.real, label="Haslam")
plt.gca().yaxis.set_ticklabels([])
plt.legend()

plt.subplot(2,1,2)
plt.plot(test_za/np.pi, test_map_vis.real, label="CHIME")
plt.gca().yaxis.set_ticklabels([])
plt.legend()

plt.xlabel(r"$\theta_k/\pi$")
#yfmt = ticker.ScalarFormatter()
#yfmt.set_powerlimits((-2,2))
#plt.gca().yaxis.set_major_formatter(yfmt)

this_ra = ephem.transit_RA(data.time[test_ind])
plt.suptitle(u"RA of {:.1f}°".format(this_ra), y=1.02, )

plt.tight_layout()

plt.savefig(plot_path + "/slice_ra{:.0f}.png".format(this_ra), dpi=300, bbox_inches='tight')

In [ ]:
plt.plot(test_map_vis.real * test_map.real)

In [ ]:
plt.plot(np.sum(test_model.M, axis=1))

## Try a different region of RA

In [ ]:
target_ra_far = (269., 271.)
target_time_far = [ ephem.transit_times(r, start_time) for r in target_ra_far ]
datar.select_time_range(*target_time_far)

In [ ]:
target_time_far

In [ ]:
data_far = datar.read()

In [ ]:
vis_cal_far = data_far.vis[0,:,:] * np.outer(gain[gain_freq_ind[f_ind],:256], 
                              gain[gain_freq_ind[f_ind],:256].conj())[np.triu_indices(256)][:,np.newaxis]
wgt_cal_far = data_far.weight[0,:,:] / np.abs(np.outer(gain[gain_freq_ind[f_ind],:256], 
                              gain[gain_freq_ind[f_ind],:256].conj())[np.triu_indices(256)][:,np.newaxis])**2

In [ ]:
bad_input_far = np.prod(data_far.flags['inputs'], axis=1)
print np.sum(bad_input_far)
bad_input_far *= gain[gain_freq_ind[f_ind],:256] != 0
print np.sum(bad_input_far)
bad_prod_far = np.outer(bad_input_far, bad_input_far)[np.triu_indices(256)]
# remove autos
bad_prod_far[np.array(autos_ind)] = 0.
# remove short baselines
#bad_prod[np.where(np.abs(ns_baselines) < 3. * 0.3)[0]] = 0.
good_prod_far = np.where(bad_prod_far != 0.)[0]

In [ ]:
vis_cal_far = vis_cal_far[good_prod]
wgt_cal_far = wgt_cal_far[good_prod]

In [ ]:
test_ind_far = vis_cal_far.shape[-1] / 2
time_slice_far = slice(test_ind_far - 20, test_ind_far + 20, 4)

In [ ]:
test_map_vis_far = np.dot(vis_cal_far[:,test_ind_far] / ns_bl_norm, np.exp(-2j * np.pi * ns_baselines[:,np.newaxis]
                                        / test_model.wl * np.sin(test_za)[np.newaxis,:]))

In [ ]:
test_model_far = ModelVis(freq=freq[f_ind])
test_model_far.set_baselines(ns_baselines)

In [ ]:
beam_sol_far = test_model_far.fit_beam(data_far.time[time_slice_far], 
                                   vis_cal_far[:,time_slice_far],
                                   wgt_cal_far[:,time_slice_far],
                                   num_pix, max_za=max_za, rcond=1e-9)

In [ ]:
beam_sol_far = test_model_far.fit_beam(data_far.time[test_ind_far:test_ind_far+1], 
                                   vis_cal_far[:,test_ind_far:test_ind_far+1],
                                   wgt_cal_far[:,test_ind_far:test_ind_far+1],
                                   num_pix, max_za=max_za, rcond=1e-9)

In [ ]:
plt.plot(test_za/np.pi, beam_sol_far, label="RA~{:.1f}".format(ephem.transit_RA(data_far.time[test_ind_far])))
plt.plot(test_za/np.pi, beam_sol_single, label="RA~{:.1f}".format(ephem.transit_RA(data.time[test_ind])))
plt.legend()

plt.ylabel(r"fit parameters $\alpha_k$")
plt.xlabel(r"$\theta_k / \pi$")

plt.title("{:.2f} MHz".format(freq[f_ind]))

plt.savefig(plot_path + "/beam_soln.png", dpi=300, bbox_inches='tight')

In [ ]:
test_vis_far = test_model_far.get_vis(data_far.time[test_ind_far:test_ind_far+1],
                                      vis_cal_far[:,test_ind_far:test_ind_far+1],
                                      num_pix, max_za=max_za)
test_map_far = np.dot(test_vis_far[:,0]/ns_bl_norm, np.exp(-2j * np.pi * ns_baselines[:,np.newaxis]
                                        / test_model.wl * np.sin(test_za)[np.newaxis,:]))

In [ ]:
plt.subplot(2,1,1)
plt.plot(test_za/np.pi, test_map_far.real, label="Haslam")
plt.gca().yaxis.set_ticklabels([])
plt.legend()

plt.subplot(2,1,2)
plt.plot(test_za/np.pi, test_map_vis_far.real, label="CHIME")
plt.gca().yaxis.set_ticklabels([])
plt.legend()

plt.xlabel(r"$\theta_k/\pi$")
#yfmt = ticker.ScalarFormatter()
#yfmt.set_powerlimits((-2,2))
#plt.gca().yaxis.set_major_formatter(yfmt)

this_ra = ephem.transit_RA(data_far.time[test_ind_far])
plt.suptitle(u"RA of {:.1f}°".format(this_ra), y=1.02, )

plt.tight_layout()

plt.savefig(plot_path + "/slice_ra{:.0f}.png".format(this_ra), dpi=300, bbox_inches='tight')

In [ ]:
ephem.transit_RA(data_far.time[test_ind_far])

## Try on multiple frequencies

In [ ]:
datar_f = andata.CorrReader(sorted(glob(acq_path + "*.h5"))[:18])
# select time and freq
datar_f.select_freq_physical(freq)
datar_f.select_time_range(*target_time)
datar_f.prod_sel = prod_sel

In [ ]:
data_f = datar_f.read()

In [ ]:
vis_cal_mf = []
wgt_cal_mf = []
for f in range(len(freq)):
    vis_cal_mf.append(data_f.vis[f,:,:] * np.outer(gain[gain_freq_ind[f],:256], 
                              gain[gain_freq_ind[f],:256].conj())[np.triu_indices(256)][:,np.newaxis])
    wgt_cal_mf.append(data_f.weight[f,:,:] / np.abs(np.outer(gain[gain_freq_ind[f],:256], 
                              gain[gain_freq_ind[f],:256].conj())[np.triu_indices(256)][:,np.newaxis])**2)

In [ ]:
bad_input_mf = np.prod(data_f.flags['inputs'], axis=1)
print np.sum(bad_input_mf)
bad_input_mf = bad_input_mf * gain[gain_freq_ind,:256] != 0
print np.sum(bad_input_far)
bad_prod_mf = np.zeros((len(freq), vis_cal_mf[0].shape[0]))
for f in range(len(freq)):
    bad_prod_mf[f] = np.outer(bad_input_mf[f], bad_input_mf[f])[np.triu_indices(256)]
# remove autos
bad_prod_mf[:,np.array(autos_ind)] = 0.
# remove short baselines
#bad_prod[np.where(np.abs(ns_baselines) < 3. * 0.3)[0]] = 0.

good_prod_mf = []
for f in range(len(freq)):
    good_prod_mf.append(np.where(bad_prod_mf[f] != 0.)[0])

In [ ]:
ns_baselines_all = ((data.prod['input_a'] % 256).astype(float)
                 - data.prod['input_b'] % 256) * 0.3

In [ ]:
ns_baselines_mf = []
for f in range(len(freq)):
    vis_cal_mf[f] = vis_cal_mf[f][good_prod_mf[f]]
    wgt_cal_mf[f] = wgt_cal_mf[f][good_prod_mf[f]]
    ns_baselines_mf.append(ns_baselines_all[good_prod_mf[f]])

In [ ]:
test_ind_mf = vis_cal_mf[0].shape[-1] / 2
time_slice_mf = slice(test_ind_mf - 2, test_ind_mf + 2)

In [ ]:
test_model_mf = []
for f in range(len(freq)):
    test_model_mf.append(ModelVis(freq=freq[f]))
    test_model_mf[-1].set_baselines(ns_baselines_mf[f])

In [ ]:
beam_sol_mf = np.zeros((len(freq), num_pix))
for f in range(1, len(freq)):
    beam_sol_mf[f] = test_model_mf[f].fit_beam(data_f.time[time_slice_mf], 
                                   vis_cal_mf[f][:,time_slice_mf],
                                   wgt_cal_mf[f][:,time_slice_mf],
                                   num_pix, max_za=max_za, rcond=1e-9)

In [ ]:
for f in range(1, len(freq)):
    plt.plot(beam_sol_mf[f], label="{:.1f}".format(freq[f]))
    
plt.legend()
plt.ylim(ymin=-0.005, ymax=0.005)